In [1]:
import numpy as np
from wNMF import wNMF

In [2]:
## An example on simulated data
n=100
features = 100
components=25
X = 100*np.random.uniform(size=n*features).reshape(features,n)
W = np.ones_like(X)

In [3]:
model = wNMF(n_components=components,beta_loss='frobenius',max_iter=1000,track_error=True,verbose=1)
fit = model.fit(X=X,W=W,n_run=5)

Beginning Run 1...
|--- Initializing U,V
|--- Running wNMF
|--- Completed
Beginning Run 2...
|--- Initializing U,V
|--- Running wNMF
|--- Completed
Beginning Run 3...
|--- Initializing U,V
|--- Running wNMF
|--- Completed
Beginning Run 4...
|--- Initializing U,V
|--- Running wNMF
|--- Completed
Beginning Run 5...
|--- Initializing U,V
|--- Running wNMF
|--- Completed


In [4]:
print(fit.V)

[[4.00860530e-27 1.31994032e+00 4.21531866e+00 ... 9.11356400e-01
  1.87781216e-29 2.44932809e+00]
 [7.83584247e-01 3.16455355e+00 6.12003608e+00 ... 7.11255701e-15
  1.03949120e-33 3.24310972e+00]
 [3.15761581e+00 7.65305913e-14 1.16601993e-24 ... 3.83710303e-10
  4.28795006e+00 1.03722827e-25]
 ...
 [1.59232464e+00 2.36455064e-22 2.83798133e+00 ... 8.57793178e-26
  4.90021998e-07 2.64733818e-14]
 [2.92701598e-22 1.83003113e+00 1.11951451e-10 ... 3.74053679e+00
  3.97628027e+00 5.16397207e-13]
 [2.23183477e+00 3.71569832e+00 1.47654180e-02 ... 1.31400958e-23
  3.60025263e-14 8.85043609e+00]]


In [5]:
print(fit.U)

[[1.41150636e+00 2.47716529e+00 2.33401600e-06 ... 1.98445882e-10
  3.54627801e+00 5.89970355e-02]
 [1.51942521e-30 6.61544116e-01 2.78943805e+00 ... 2.35617928e+00
  1.14599971e-21 1.32561970e+00]
 [4.27043340e-13 1.78904997e+00 4.78235108e-16 ... 2.71290018e-03
  2.06168524e-45 9.93047307e-01]
 ...
 [3.07327269e-29 3.44642062e+00 4.67635580e+00 ... 4.38505241e+00
  5.19588534e-66 4.91520809e-07]
 [2.57904726e+00 7.13217287e-27 2.55943563e+00 ... 2.65270046e-01
  2.70765091e+00 1.49812518e+00]
 [7.09929424e-13 5.52095277e+00 1.86796289e-23 ... 4.70131849e-37
  1.45421180e-01 3.75713809e+00]]


In [6]:
print(fit.err)

1935747.875830764


In [7]:
import pandas as pd
errdf = pd.DataFrame(np.log10(fit.error_tracker)).T.rename({i:'run_'+str(i) for i in range(0,components)},axis='columns')

In [8]:
import altair as alt
alt.Chart(errdf.loc[100:,:].reset_index().melt(id_vars='index')).mark_point().encode(
    alt.X('index:Q'),
    alt.Y('value:Q',scale=alt.Scale(zero=False)),
    color='variable'  
)

alt.Chart(...)